In [1]:
#import what we need
from requests_html import HTMLSession
session = HTMLSession()
import pandas as pd
import newspaper
import json
import geopandas as gpd


#use session to get the page
r = session.get('https://news.google.com/search?q=farm%20loss%20in%20Telangana&hl=en-IN&gl=IN&ceid=IN%3Aen')  #agriculture telangana

#render the html, sleep=1 to give it a second to finish before moving on. scrolldown= how many times to page down on the browser, to get more results. 5 was a good number here
#r.html.render(sleep=1, scrolldown=5)     # ----------------------# THIS LINE WAS FETCHING ME ERROR IN JUPYTER NOTEBOOK

#find all the articles by using inspect element and create blank list
articles = r.html.find('article')
newslist = []

#loop through each article to find the title and link. try and except as repeated articles from other sources have different h tags.
for item in articles:
    try:
        newsitem = item.find('h3', first=True)
        title = newsitem.text
        link = newsitem.absolute_links
        newsarticle = {
            'title': title,
            'link': link 
        }
        newslist.append(newsarticle)
    except:
       pass

#print the length of the list
print('List of news articles: ',len(newslist))

List of news articles:  100


In [3]:
#Create a dataframe of newslist and link
df = pd.DataFrame(newslist)


#Scrape the list of article dictionary from news list
news_articles = []
for i in df.link:
    url = list(i)[0]
    article = newspaper.Article(url=url, language='en')
    article.download()
    article.parse()

    article ={
        "title": str(article.title),
        "text": str(article.text),
        "authors": article.authors,
        "published_date": str(article.publish_date),
        "top_image": str(article.top_image),
        "videos": article.movies,
        "keywords": article.keywords,
        "summary": str(article.summary)
    }
    news_articles.append(article)

In [4]:
#Convert list of news articles into dataframe with all scraped information
news = pd.DataFrame(news_articles)
text = news.text[0]

In [5]:
#Read geodataframe for mandal and district names to search in news articles
gdf = gpd.read_file('../DPPD/data/TSDM/Mandal_Boundary.shp')
mandal = list(gdf.Mandal_Nam.unique()) #unique mandals
dist = list(gdf.Dist_Name.unique())    #unique districts

In [6]:
# Access list of mandal mentione in each article

keyword_mandal = []
mandal_list = []

for text in  news.text:
    for i in range(0, len(mandal)):
        if mandal[i] in text:
            keyword_mandal.append(mandal[i])
    # insert the list to the set
        list_set = set(keyword_mandal)
    # convert the set to the list
        unique_list = (list(list_set))
    mandal_list.append(unique_list)


In [7]:
# Access list of district mentione in each article
keyword_dist = []
dist_list = []
for text in  news.text:
    for i in range(0, len(dist)):
        if dist[i] in text:
            keyword_dist.append(dist[i])
    # insert the list to the set
        list_set = set(keyword_dist)
    # convert the set to the list
        unique_list = (list(list_set))
    dist_list.append(unique_list)

In [8]:
#add mandal and districts to news dataframe
news['Mandals'] = mandal_list
news['Districts'] = dist_list

#convert full news dataframe to csv
news.to_csv('news_full.csv', index = False)

In [9]:
#Dataframe with few columns
news1 = news[['title','Mandals','Districts','published_date']]
news1.to_csv('news.csv', index = False)

In [11]:
news

,title,text,authors,published_date,top_image,videos,keywords,summary,Mandals,Districts
0,From paddy to palm oil: Telangana focuses on w...,"After a push for paddy production, the Telanga...",[],2022-12-06 10:43:49+00:00,https://imgs.mongabay.com/wp-content/uploads/s...,[],[],,[Suryapet],"[Hyderabad, Khammam, Suryapet]"
1,‘Telangana’s agricultural output has increased...,Hyderabad: Telangana Agriculture minister S Ni...,[News Desk],2022-11-25 13:40:53+00:00,https://cdn.siasat.com/wp-content/uploads/2022...,[],[],,[Suryapet],"[Hyderabad, Khammam, Suryapet]"
2,Telangana: Agricultural residential degree col...,Telangana: Agricultural residential degree col...,[Telangana Today],2022-12-03 20:54:43+05:30,https://cdn.telanganatoday.com/wp-content/uplo...,[],[],,"[Karimnagar, Suryapet, Wanaparthy]","[Khammam, Wanaparthy, Hyderabad, Karimnagar, S..."
3,Telangana sets new benchmarks in agricultural ...,Telangana sets new benchmarks in agricultural ...,[Telangana Today],2022-11-11 20:43:53+05:30,https://cdn.telanganatoday.com/wp-content/uplo...,[],[],,"[Karimnagar, Suryapet, Wanaparthy]","[Khammam, Wanaparthy, Hyderabad, Karimnagar, S..."
4,Telangana govt to provide free EAMCET coaching...,Hyderabad: Telangana government is getting rea...,[Sameer Khan],2022-12-06 05:40:45+00:00,https://cdn.siasat.com/wp-content/uploads/2022...,[],[],,"[Karimnagar, Suryapet, Wanaparthy]","[Khammam, Wanaparthy, Hyderabad, Karimnagar, S..."
...,...,...,...,...,...,...,...,...,...,...
95,Telangana International Seed Testing Lab gets ...,Telangana International Seed Testing Lab gets ...,[Telangana Today],2022-08-27 23:58:51+05:30,https://cdn.telanganatoday.com/wp-content/uplo...,[],[],,"[Gadwal, Dichpally, Doulthabad, Rajendranagar,...","[Siddipet, Nizamabad, Jogulamba Gadwal, Rajann..."
96,Malnutrition: Telangana’s efforts find place i...,Malnutrition: Telangana’s efforts find place i...,[Telangana Today],2022-05-24 22:11:19+05:30,https://cdn.telanganatoday.com/wp-content/uplo...,[],[],,"[Gadwal, Dichpally, Doulthabad, Rajendranagar,...","[Siddipet, Nizamabad, Jogulamba Gadwal, Rajann..."
97,Paddy cultivation in Telangana goes up by 7 pe...,| Paddy Cultivation In Telangana Goes Up By 7 ...,[Telangana Today],2022-08-26 21:49:29+05:30,https://cdn.telanganatoday.com/wp-content/uplo...,[],[],,"[Gadwal, Dichpally, Doulthabad, Rajendranagar,...","[Siddipet, Nizamabad, Jogulamba Gadwal, Rajann..."
98,Telangana minister Singireddy Niranjan Reddy t...,By Express News Service\n\nKARIMNAGAR: Telanga...,[],None,https://images.newindianexpress.com/uploads/us...,[],[],,"[Gadwal, Dichpally, Doulthabad, Rajendranagar,...","[Siddipet, Nizamabad, Jogulamba Gadwal, Rajann..."


In [13]:
newslistdf = pd.DataFrame(newslist)

In [15]:
newslistdf.to_csv('newslist.csv', index=False)

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import newspaper
import geopandas as gpd

from GoogleNews import GoogleNews
googlenews = GoogleNews()

In [2]:
googlenews = GoogleNews()
googlenews.headers = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Cafari/537.36'
googlenews = GoogleNews(start='01/12/2022',end='07/12/2022')
googlenews.search('agriculture Khammam')
   
result=googlenews.result()
df=pd.DataFrame(result)

HTTP Error 429: Too Many Requests


In [3]:
import geopandas as gpd

#Read geodataframe for mandal and district names to search in news articles
gdf = gpd.read_file('dicra/analytics/sandbox/notebooks/SoilMoistureDeviance/Results/')
mandal = list(gdf.Mandal_Nam.unique()) #unique mandals
dist = list(gdf.Dist_Name.unique())    #unique districts

DriverError: dicra/analytics/sandbox/notebooks/SoilMoistureDeviance/Results/: No such file or directory